In [20]:
import numpy as np
import pandas as pd

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
color = sns.color_palette()
sns.set()

%matplotlib inline

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from funcs import correct_macro_df, get_corr_empty_info#, prepare_choosed_features

pd.options.mode.chained_assignment = None # default='warn'
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [77]:
train_df = pd.read_csv("data/train.csv", parse_dates=['timestamp'])
test_df = pd.read_csv("data/test.csv", parse_dates=['timestamp'])

macro_df = pd.read_csv("data/macro.csv", parse_dates=['timestamp'])
macro_df = correct_macro_df(macro_df=macro_df)
macro_df.columns = ["timestamp"] + ["macro_" + c for c in macro_df.columns if c!="timestamp"]

train_with_macro_df = pd.merge(train_df, macro_df, how='left', on='timestamp')
test_with_macro_df = pd.merge(test_df, macro_df, how='left', on='timestamp')

In [78]:
def add_dates_info(df):
    df = df.copy()
    df["timestamp_dt"] = pd.to_datetime(df["timestamp"])
    df["timestamp_year"] = df["timestamp_dt"].apply(lambda x: int(x.year))
    df["timestamp_year_month"] = df["timestamp_year"].astype(str) + df["timestamp_dt"].apply(lambda x: str(x.month))
    return df

train_with_macro_df = add_dates_info(train_with_macro_df)
test_with_macro_df = add_dates_info(test_with_macro_df)

In [79]:
macro_with_avg_price_df = train_with_macro_df[
    ['timestamp_year_month'] + list(filter(lambda x: "macro_" in x, train_with_macro_df.columns))]

In [80]:
macro_with_avg_price_gb = macro_with_avg_price_df.groupby(['timestamp_year_month'])

In [84]:
macro_with_avg_price_gb.get_group("201110")

,timestamp_year_month,macro_oil_urals,macro_gdp_quart,macro_gdp_quart_growth,macro_cpi,macro_ppi,macro_gdp_deflator,macro_balance_trade,macro_balance_trade_growth,macro_usdrub,macro_eurrub,macro_brent,macro_net_capital_export,macro_gdp_annual,macro_gdp_annual_growth,macro_average_provision_of_build_contract,macro_average_provision_of_build_contract_moscow,macro_rts,macro_micex,macro_micex_rgbi_tr,macro_micex_cbi_tr,macro_deposits_value,macro_deposits_growth,macro_deposits_rate,macro_mortgage_value,macro_mortgage_growth,macro_mortgage_rate,macro_grp,macro_grp_growth,macro_income_per_cap,macro_real_dispos_income_per_cap_growth,macro_salary,macro_salary_growth,macro_fixed_basket,macro_retail_trade_turnover,macro_retail_trade_turnover_per_cap,macro_retail_trade_turnover_growth,macro_labor_force,macro_unemployment,macro_employment,macro_invest_fixed_capital_per_cap,macro_invest_fixed_assets,macro_profitable_enterpr_share,macro_unprofitable_enterpr_share,macro_share_own_revenues,macro_overdue_wages_per_cap,macro_fin_res_per_cap,macro_marriages_per_1000_cap,macro_divorce_rate,macro_construction_value,macro_invest_fixed_assets_phys,macro_pop_natural_increase,macro_pop_migration,macro_pop_total_inc,macro_childbirth,macro_mortality,macro_housing_fund_sqm,macro_lodging_sqm_per_cap,macro_water_pipes_share,macro_baths_share,macro_sewerage_share,macro_gas_share,macro_hot_water_share,macro_electric_stove_share,macro_heating_share,macro_old_house_share,macro_average_life_exp,macro_infant_mortarity_per_1000_cap,macro_perinatal_mort_per_1000_cap,macro_incidence_population,macro_rent_price_4+room_bus,macro_rent_price_3room_bus,macro_rent_price_2room_bus,macro_rent_price_1room_bus,macro_rent_price_3room_eco,macro_rent_price_2room_eco,macro_rent_price_1room_eco,macro_load_of_teachers_preschool_per_teacher,macro_child_on_acc_pre_school,macro_load_of_teachers_school_per_teacher,macro_students_state_oneshift,macro_modern_education_share,macro_old_education_build_share,macro_provision_doctors,macro_provision_nurse,macro_load_on_doctors,macro_power_clinics,macro_hospital_beds_available_per_cap,macro_hospital_bed_occupancy_per_year,macro_provision_retail_space_sqm,macro_provision_retail_space_modern_sqm,macro_turnover_catering_per_cap,macro_theaters_viewers_per_1000_cap,macro_seats_theather_rfmin_per_100000_cap,macro_museum_visitis_per_100_cap,macro_bandwidth_sports,macro_population_reg_sports_share,macro_students_reg_sports_share,macro_apartment_build,macro_apartment_fund_sqm
42,201110,108.24,15663.6,5.0,353.0,436.5,86.721,17.199,18.1,32.1840,43.2804,102.76,0.447362,46308.5,0.045037,5.96,6.65,1341.09,1366.54,128.65,204.21,10920215,0.01859,4.5,452674,1.030292,11.57,9948.7728,0.187791,44388.0,-0.005,44898.7,0.168917,12922.97,3322.047,286.952,106.6,6643.626,0.014,0.708,73976.19863,856.424079,0.708,0.292,0.891478,53636.0,226.214157,8.5,3.8,549075.8,106.6,1.1,5.1,6.2,10.8,9.7,218.0,18.772066,99.9,99.8,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,151.68,81.5,63.84,52.25,43.46,38.1,31.11,793.319561,NaN,1391.710938,89.0495,NaN,NaN,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
43,201110,108.24,15663.6,5.0,353.0,436.5,86.721,17.199,18.1,32.1840,43.2804,102.76,0.447362,46308.5,0.045037,5.96,6.65,1341.09,1366.54,128.65,204.21,10920215,0.01859,4.5,452674,1.030292,11.57,9948.7728,0.187791,44388.0,-0.005,44898.7,0.168917,12922.97,3322.047,286.952,106.6,6643.626,0.014,0.708,73976.19863,856.424079,0.708,0.292,0.891478,53636.0,226.214157,8.5,3.8,549075.8,106.6,1.1,5.1,6.2,10.8,9.7,218.0,18.772066,99.9,99.8,99.5,43.9,95.7,55.3,99.9,0.3,75.79,6.2,5.53,715.1,151.68,81.5,63.84,52.25,43.46,38.1,31.11,793.319561,NaN,1391.710938,89.0495,NaN,NaN,65.9,99.6,8180.755454,375.8,846.0,302.0,741.0,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
44,201110,108.24,15663.6,5.0,353.0,436.5,86.721,17.199,18.1,32.1840,43.2804,102.76,0.447362,46308.5,0.045037,5.96,6.65,1341.09,1366.54,128.65,2

In [68]:
macro_with_avg_price_df_corr = macro_with_avg_price_df.corr()
macro_with_avg_price_df_corr = macro_with_avg_price_df_corr.where(
    ~np.triu(np.ones(macro_with_avg_price_df_corr.shape)).astype(np.bool)).stack().reset_index()
macro_with_avg_price_df_corr.columns = ["feature1", "feature2", "corr_value"]
# macro_with_avg_price_df_corr.corr_value = macro_with_avg_price_df_corr.corr_value.round(9)

In [69]:
macro_with_avg_price_df_corr.where(
    macro_with_avg_price_df_corr.feature2=="avg_month_price_per_sqm").dropna().corr_value.round(8)

Series([], Name: corr_value, dtype: float64)

In [ ]:
train_gb = train_df.groupby(['timestamp_year_month'])

dfagg = pd.DataFrame()
dfagg['avg_price_per_sqm'] = train_gb.price_doc.sum() / train_gb.full_sq.sum()
dfagg['rolling_average_immo'] = dfagg['avg_price_per_sqm'].rolling(30).mean()
dfagg.reset_index(inplace=True)

In [217]:
# gdp_annual
# gdp_annual_growth
# gdp_deflator
# gdp_quart
# gdp_quart_growth

# grp
# grp_growth

# cpi
# ppi

# oil_urals
# usdrub
# eurrub

# micex
# micex_cbi_tr
# micex_rgbi_tr

# rent_price_1room_bus
# rent_price_1room_eco
# rent_price_2room_bus
# rent_price_2room_eco
# rent_price_3room_bus
# rent_price_3room_eco
# rent_price_4+room_bus

# apartment_build
# apartment_fund_sqm
# average_life_exp
# average_provision_of_build_contract
# average_provision_of_build_contract_moscow
# balance_trade
# balance_trade_growth
# bandwidth_sports
# baths_share
# brent
# child_on_acc_pre_school
# childbirth
# construction_value
# date
# deposits_growth
# deposits_rate
# deposits_value
# divorce_rate
# electric_stove_share
# employment
# fin_res_per_cap
# fixed_basket
# gas_share
# heating_share
# hospital_bed_occupancy_per_year
# hospital_beds_available_per_cap
# hot_water_share
# housing_fund_sqm
# incidence_population
# income_per_cap
# infant_mortarity_per_1000_cap
# invest_fixed_assets
# invest_fixed_assets_phys
# invest_fixed_capital_per_cap
# labor_force
# load_of_teachers_preschool_per_teacher
# load_of_teachers_school_per_teacher
# load_on_doctors
# lodging_sqm_per_cap
# marriages_per_1000_cap
# modern_education_share
# mortality
# mortgage_growth
# mortgage_rate
# mortgage_value
# museum_visitis_per_100_cap
# net_capital_export
# old_education_build_share
# old_house_share
# overdue_wages_per_cap
# perinatal_mort_per_1000_cap
# pop_migration
# pop_natural_increase
# pop_total_inc
# population_reg_sports_share
# power_clinics
# profitable_enterpr_share
# provision_doctors
# provision_nurse
# provision_retail_space_modern_sqm
# provision_retail_space_sqm
# real_dispos_income_per_cap_growth

# retail_trade_turnover
# retail_trade_turnover_growth
# retail_trade_turnover_per_cap
# rts
# salary
# salary_growth
# seats_theather_rfmin_per_100000_cap
# sewerage_share
# share_own_revenues
# students_reg_sports_share
# students_state_oneshift
# theaters_viewers_per_1000_cap
# timestamp
# turnover_catering_per_cap
# unemployment
# unprofitable_enterpr_share
# water_pipes_share

In [23]:
# first let's average per day
gb_train = train_df.groupby(['timestamp'])
dfagg = pd.DataFrame()
dfagg['avg_price_per_sqm'] = gb_train.price_doc.sum() / gb_train.full_sq.sum()
dfagg['rolling_average_immo'] = dfagg['avg_price_per_sqm'].rolling(30).mean()
dfagg.reset_index(inplace=True)
dfagg = pd.merge(dfagg, macro_df, how='left', on=['timestamp'])

In [35]:
macro_df.columns

Index(['timestamp', 'oil_urals', 'gdp_quart', 'gdp_quart_growth', 'cpi', 'ppi',
       'gdp_deflator', 'balance_trade', 'balance_trade_growth', 'usdrub',
       ...
       'turnover_catering_per_cap', 'theaters_viewers_per_1000_cap',
       'seats_theather_rfmin_per_100000_cap', 'museum_visitis_per_100_cap',
       'bandwidth_sports', 'population_reg_sports_share',
       'students_reg_sports_share', 'apartment_build', 'apartment_fund_sqm',
       'date'],
      dtype='object', length=101)